In [4]:
import color_correlation
import cv2
import dewarper
import unsharpen

In [22]:
testRect = {
    "audit_type": "rect",
    "id": 27,
    "page_number": 1,
    "template_signatory_id": 10,
    "x1": 0.122898037659841,
    "x2": 0.169011505478072,
    "y1": 0.912442396313364,
    "y2": 0.943205053865169
}

In [7]:
source_image = cv2.imread("files/VacantLandContract - Signed [1] (1)-0.png")


In [9]:
source_image.shape

(792, 612, 3)

In [28]:
def translate_coords(coord1, coord2, span):
    return [int(coord1 * span), int(coord2 * span)]
    

In [32]:
cropped = source_image[slice(*translate_coords(testRect['y1'], testRect['y2'], source_image.shape[0])), slice(*translate_coords(testRect['x1'], testRect['x2'], source_image.shape[1]))]

In [34]:
cropped.shape

(25, 28, 3)